<a href="https://colab.research.google.com/github/wafibismail/davis-busroutes/blob/main/Davis_group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# for plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.offline import iplot # producing plot htmls
from scipy.interpolate import griddata # surface plots

# for evaluating models
from sklearn.metrics import mean_squared_error

# data manipulation
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn import metrics

# for ANN
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# regressors
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

# For generating javascript function of decision tree regressor
from sklearn.tree import _tree

In [24]:
# Configs
# Evaluation metrics will be averaged over the following number of runs:
ANN_runs = 20
reg_runs = 40
# On the run number stated below, show the true values vs prediction comparison
special_seed_val = 5
# Where the raw dataset files are to be obtained from
raw_dir = 'https://raw.githubusercontent.com/wafibismail/davis-busroutes/main/raw_dataset/'


In [25]:
# PREPARE AND CLEAN DATA

# Initialize array containing day of the week of each date
# 1st October 2023 is Sunday
# Let {0, 1, ... , 6} represent {Sunday, Monday, Tuesday, ... , Saturday}
available_dates = np.array([1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 15, 16, 17, 18,
                           19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])
days = (available_dates-1)%7

# initialize a list containing each dataframe containing timestamps for each day
df_list = []
for n in range(len(available_dates)):
  df_list.append(
      pd.read_csv(raw_dir+'2023-10-'+f'{available_dates[n]:02}'+'.csv'))

# function to convert time strings to durations in seconds, one column at a time
def to_seconds(column):
  time = pd.to_datetime(column, format= '%H:%M:%S' )
  time = time.fillna(datetime(2024,1,1)) # assign NaN -> 0 for ease of processing
  return (time.dt.hour*60+time.dt.minute)*60 + time.dt.second

# produce a list of dataframes that contain only malleable time information
for i in range(len(df_list)): # loop thru each dataframe
  # Drop redundant / unusable columns
  df_list[i] = df_list[i].drop(labels={
      "bus_id","route_id","place1","place2","place3",
      "place4","place5","place6","place7","place8","place9"}, axis=1)

  for j in range(df_list[i].shape[1]): # loop thru each column of each dataframe
    # convert the time strings to total duration in seconds
    df_list[i][df_list[i].columns[j]] = \
      to_seconds(df_list[i][df_list[i].columns[j]])

# Instantiate dictionary to contain the main DataFrame columns:
t_dict = {
    'path#': [],
    'path': [],
    'day#': [],
    'day': [],
    'time#': [],
    'time': [],
    'idle_duration': [],
    'move_duration': [],
    'previous_move_duration': [],
    'previous_previous_move_duration': []}
#  path#: number representing coding of the column described next
#  path: label of travel from THIS to NEXT station, e.g. 1->2, 2->3, ..., 8->9
#  day#: number representing day of the week - 0 to 9
#  day : day of the week - Sunday, Monday, Tuesday, ..., Saturday
#  time: time e.g. 8 o'clock, but in seconds e.g. 28800
#  idle_duration: amount of time spent in THIS station
#  move_duration: amount of time spent travelling from THIS to NEXT station
#  previous_move_duration: same as above, but the preceding adjacent record
#  previous_previous_move_duration: similar to above, self-explanatory

# Initialize lists containing day and path names, limits of time discretization
day_names = ['Sunday', 'Monday', 'Tuesday', \
             'Wednesday', 'Thusday', 'Friday', 'Saturday']
timebreaks_source = [6, 8, 10, 12, 13.5, 16, 17.5, 19.5, 21.5, 23.25, 24]
timebreaks = [(timebreaks_source[i+1]*60*60) for i in range(10)]
pathLabels = [(str(i+1) + " -> " + str(i+2)) for i in range(8)]

initial_row_count = 0 # how many rows in all csv files combined

for i in range(len(df_list)):
  n_rows = df_list[i].shape[0]
  for j in range(n_rows):
    t_in = df_list[i].iloc[j,0::2].values
    t_out = df_list[i].iloc[j,1::2].values
    initial_row_count += 1

    for k in range(8):
      if all(t_in[k+0:k+2]) and all(t_out[k:k+2]):

        # Initialize values to be added as the "cleaned" data
        pathNo = k
        pathLabel = pathLabels[k]
        dayNo = days[i]
        day = day_names[dayNo]
        time = np.average([t_in[k+1], t_out[k]])
        # ^ average timestamp between time_in and time_out of one station
        idleDuration = t_out[k]-t_in[k]
        # ^ time out of this station - time in of this station
        moveDuration = t_in[k+1]-t_out[k]
        # ^ time in of next station - time out of this station
        prevMoveDuration = 0
        # ^ time in of this station - time out of previous station
        prevPrevMoveDuration = 0
        # ^ time in of previous station - time out of previous^2 station

        # Compute preceding busstop values, leave as 0 if invalid
        if k > 0:
          if all(t_in[k-1:k+1]) and all(t_out[k-1:k+1]):
            prevMoveDuration = t_in[k]-t_out[k-1]
        if k > 1:
          if all(t_in[k-2:k+1]) and all(t_out[k-2:k+1]):
            prevPrevMoveDuration = t_in[k-1]-t_out[k-2]

        # Append clean data into dictionary
        t_dict['path#'].append(pathNo)
        t_dict['path'].append(pathLabel)
        t_dict['day#'].append(dayNo)
        t_dict['day'].append(day)
        t_dict['time'].append(time)
        for l in range(len(timebreaks)):
          if time < timebreaks[l]:
            t_dict['time#'].append(l)
            break
        t_dict['idle_duration'].append(idleDuration)
        t_dict['move_duration'].append(moveDuration)
        t_dict['previous_move_duration'].append(prevMoveDuration)
        t_dict['previous_previous_move_duration'].append(prevPrevMoveDuration)

# Create a dataframe out of the prepared data
# Sort it according to move and day#
t_df = pd.DataFrame(data=t_dict).sort_values(by=['path#', 'day#'], ignore_index=True)

# Print number of records before and after cleaning & further processing
print(initial_row_count, "Total number of rows before cleaning & distilling durations from raw dataset")
print(len(t_df), "Total number of records after cleaning")
print(len(t_df.loc[t_df["previous_move_duration"]>0]), "Total number of clean records that has valid corresponding dircetly preceding record")
print(len(t_df.loc[t_df["previous_previous_move_duration"]>0]), "Total number of clean records that has 2 valid corresponding dircetly preceding adjacent records")

270 Total number of rows before cleaning & distilling durations from raw dataset
1209 Total number of records after cleaning
828 Total number of clean records that has valid corresponding dircetly preceding record
497 Total number of clean records that has 2 valid corresponding dircetly preceding adjacent records


In [26]:
# Central Tendency i.e., mean
grp = t_df.groupby("path").agg({"move_duration":{"mean"}})
grp.columns =  ["_".join(col) for col in grp.columns]
grp = grp.reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=grp["path"],
                         y=grp["move_duration_mean"],
                         mode="markers",
                         showlegend=False,
                         marker=dict(symbol="line-ew-open",
                                     color="grey",
                                     line=dict(width=5),
                                     size=40)))
fig.update_layout(
    title="Mean Travel Durations by Path", title_x=0.5,
    yaxis_title="Duration (seconds)",
    xaxis_title="Path (from-to bus stops)")
fig.update_layout(
    font_family="Times New Roman",
    font_size=30)
fig.show()

grp
fig.write_html('means.html', auto_open=False)

In [27]:
fig = go.Figure()
for i in range(8):
  fig.add_trace(go.Box(
      x=t_df.loc[t_df["path#"]==i]["move_duration"],
      name=pathLabels[i],
      marker=dict(size=8)))
fig.update_layout(
    title="Box Plot of Travel Durations grouped by Path", title_x=0.5,
    xaxis_title="Duration (seconds)",
    yaxis_title="Path (from-to bus stops)")
fig.update_layout(
    font_family="Times New Roman",
    font_size=30)
fig.show()
fig.write_html('boxplot.html', auto_open=False)

In [28]:
# Surface plot (duration against path and day)
grp = t_df.groupby(["path#", "day#"]).agg({"move_duration":{"mean"}})
grp.columns =  ["_".join(col) for col in grp.columns]
grp = grp.reset_index()

fig = go.Figure()
x = grp["path#"]
y = grp["day#"]
z = grp["move_duration_mean"]
X,Y = np.meshgrid(grp["path#"],grp["day#"])
Z = griddata((x,y),z,(X,Y), method='cubic')
fig.add_trace(go.Surface(x=x,
                         y=y,
                         z=Z))
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(
    scene = dict(
        #zaxis = dict(range=[-1000,5000]),
        aspectratio = dict(x=1,y=1,z=0.5),
        xaxis = dict(
          tickmode = 'array',
          tickvals = [0, 1, 2, 3, 4, 5, 6, 7],
          ticktext = pathLabels),
        yaxis = dict(
          tickmode = 'array',
          tickvals = [0, 1, 2, 3, 4, 5, 6],
          ticktext = day_names),
        xaxis_title='Path',
        yaxis_title='',
        zaxis_title='Avg Duration (Seconds)'
    )
)
fig.update_layout(
    font_family="Times New Roman",
    font_size=13,
    title=dict(text="Path & Day", font=dict(size=50), automargin=True, yref='paper'))
fig.show()
fig.write_html('pva_path_day.html', auto_open=False)

In [29]:
# Surface plot (duration against path and time)
grp = t_df.groupby(["path#", "time#"]).agg({"move_duration":{"mean"}})
grp.columns =  ["_".join(col) for col in grp.columns]
grp = grp.reset_index()

fig = go.Figure()
x = grp["path#"]
y = grp["time#"]
z = grp["move_duration_mean"]
X,Y = np.meshgrid(grp["path#"],grp["time#"])
Z = griddata((x,y),z,(X,Y), method='cubic')
fig.add_trace(go.Surface(x=x,
                         y=y,
                         z=Z))
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(
    scene = dict(
        #zaxis = dict(range=[-1000,5000]),
        aspectratio = dict(x=1,y=1,z=0.5),
        xaxis = dict(
          tickmode = 'array',
          tickvals = [0, 1, 2, 3, 4, 5, 6, 7],
          ticktext = pathLabels),
        xaxis_title='Path',
        yaxis_title='Time (index)',
        zaxis_title='Avg Duration (Seconds)'
    )
)
#camera = dict(
#    center=dict(x=0, y=0, z=-0.4))
#fig.update_layout(scene_camera=camera)
fig.update_layout(
    font_family="Times New Roman",
    font_size=13,
    title=dict(text="Path & Time", font=dict(size=50), automargin=True, yref='paper'))
fig.show()
fig.write_html('pva_path_time.html', auto_open=False)

In [30]:
# Surface plot (duration against day and time)
grp = t_df.groupby(["day#", "time#"]).agg({"move_duration":{"mean"}})
grp.columns =  ["_".join(col) for col in grp.columns]
grp = grp.reset_index()

fig = go.Figure()
x = grp["day#"]
y = grp["time#"]
z = grp["move_duration_mean"]
X,Y = np.meshgrid(grp["day#"],grp["time#"])
Z = griddata((x,y),z,(X,Y), method='cubic')
fig.add_trace(go.Surface(x=x,
                         y=y,
                         z=Z))
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(
    scene = dict(
        #zaxis = dict(range=[-1000,5000]),
        aspectratio = dict(x=1,y=1,z=0.5),
        xaxis = dict(
          tickmode = 'array',
          tickvals = [0, 1, 2, 3, 4, 5, 6],
          ticktext = day_names),
        xaxis_title='',
        yaxis_title='Time (index)',
        zaxis_title='Avg Duration (Seconds)'
    )
)
fig.update_layout(
    font_family="Times New Roman",
    font_size=13,
    title=dict(text="Time & Day", font=dict(size=50), automargin=True, yref='paper'))
fig.show()
fig.write_html('pva_time_day.html', auto_open=False)

In [31]:
fig = px.scatter(t_df, x="path", y="idle_duration",
           color="path", title="Idle Durations")
fig.update_traces(marker=dict(size=12),
                  selector=dict(mode='markers'))
fig.update_layout(
    font_family="Times New Roman",
    font_size=25)
fig.show()
fig.write_html('idle_durations.html', auto_open=False)

fig = px.scatter(t_df, x="path", y="move_duration",
           color="path", title="Move Durations")
fig.write_html('move_durations.html', auto_open=False)
fig.update_traces(marker=dict(size=12),
                  selector=dict(mode='markers'))
fig.update_layout(
    font_family="Times New Roman",
    font_size=25)
fig.show()
fig.write_html('move_durations.html', auto_open=False)

In [32]:
# Scatter plots showing pattern of whole dataset grouped by time and duration
#   before and after discretizing time
fig = px.scatter(t_df, x = 'time', y = 'move_duration', color = 'path')

fig.update_traces(marker=dict(size=12),
                  selector=dict(mode='markers'))
fig.update_layout(
    font_family="Times New Roman",
    font_size=25)
fig.show()
fig.write_html('time_undiscretized.html', auto_open=False)

fig = px.scatter(t_df, x = 'time#', y = 'move_duration', color = 'path')

fig.update_traces(marker=dict(size=12),
                  selector=dict(mode='markers'))
fig.update_layout(
    font_family="Times New Roman",
    font_size=25)
fig.show()
fig.write_html('time_discretized.html', auto_open=False)

In [33]:
# Function to display prediction vs actual values (for any of the 3 regressors)
def show_pred_vs_actual(regressor, X, y, regressor_name):
  y_pred = regressor.predict(X)

  dataset = np.concatenate((X,np.reshape(y, (len(y), -1))), axis=1)
  dataset = np.concatenate((dataset,np.reshape(y_pred, (len(y_pred), -1))), axis=1)

  dataset = dataset[dataset[:, 0].argsort()]

  X = dataset[:, :X.shape[1]]
  y = dataset[:, X.shape[1]]
  y_pred = dataset[:, X.shape[1]+1]

  results_dict = {
      "index": [],
      "move_duration": [],
      "origin": []
  }
  results_dict["index"] = (2*[*range(len(y))])
  results_dict["move_duration"] = (y.tolist() + \
                                  y_pred.tolist())
  results_dict["origin"] = (len(y)*["Actual data"] + (len(y))*["Prediction"])
  results_df = pd.DataFrame(data=results_dict)

  fig = go.Figure()
  pred_df = results_df.loc[results_df['origin'] == "Prediction"]
  actual_df = results_df.loc[results_df['origin'] == "Actual data"]
  fig.add_trace(go.Scatter(x=actual_df["index"],
                          y=actual_df["move_duration"],
                          mode="markers",
                          showlegend=False,
                          marker=dict(symbol="circle",
                                      color="blue",
                                      line=dict(width=0),
                                      size=9)))
  fig.add_trace(go.Scatter(x=pred_df["index"],
                          y=pred_df["move_duration"],
                          mode="markers",
                          showlegend=False,
                          marker=dict(symbol="x-thin-open",
                                      color="red",
                                      line=dict(width=1.5),
                                      size=9)))

  fig.update_layout(
      title=regressor_name, title_x=0.5,
      xaxis_title="Sorted From-To Bus Stop Test Samples")
  fig.update_layout(
    font_family="Times New Roman",
    font_size=30)
  fig.show()
  fig.write_html(regressor_name+' prediction.html', auto_open=False)
  print("R-squared",np.corrcoef(y, y_pred)[0,1]**2)
  print("RMSE",np.sqrt(mean_squared_error(y, y_pred)))

In [34]:
# Function to display prediction vs actual values (for ANN)
def show_pred_vs_actual_ANN(model, X, y, model_name):
  y_pred = model.predict(X)
  results_dict = {
      "index": [],
      "move_duration": [],
      "origin": []
  }
  results_dict["index"] = (2*[*range(len(y))])
  results_dict["move_duration"] = (y.tolist() + \
                                  y_pred[:,0].tolist())
  results_dict["origin"] = (len(y)*["Actual data"] + (len(y))*["Prediction"])
  results_df = pd.DataFrame(data=results_dict)

  fig = go.Figure()
  pred_df = results_df.loc[results_df['origin'] == "Prediction"]
  actual_df = results_df.loc[results_df['origin'] == "Actual data"]
  fig.add_trace(go.Scatter(x=actual_df["index"],
                          y=actual_df["move_duration"],
                          mode="markers+lines",
                          showlegend=False,
                          line=dict(width=1),
                          marker=dict(symbol="circle",
                                      color="blue",
                                      line=dict(width=0),
                                      size=9)))
  fig.add_trace(go.Scatter(x=pred_df["index"],
                          y=pred_df["move_duration"],
                          mode="markers+lines",
                          showlegend=False,
                          line=dict(width=1),
                          marker=dict(symbol="x-thin-open",
                                      color="red",
                                      line=dict(width=1),
                                      size=9)))

  fig.update_layout(
      title=model_name, title_x=0.5,
      xaxis_title="Random Test Samples",
      yaxis_title="Duration (seconds)")
  fig.update_layout(
    font_family="Times New Roman",
    font_size=30)
  fig.show()
  fig.write_html(model_name+' prediction.html', auto_open=False)
  print("R-squared",np.corrcoef(y, y_pred[:,0])[0,1]**2)
  print("RMSE",np.sqrt(mean_squared_error(y, y_pred)))

In [35]:
# Extract independent & dependent variables from DataFrame into X and Y
# For regression models (DT, RF, KNN)
X = np.array(t_df.iloc[:, [0, 2, 4]])
y = np.array(t_df.iloc[:, 7]).ravel()
# For trigram prediction model (ANN)
X_trigram = np.array(t_df.loc[t_df['previous_previous_move_duration'] > 0].iloc[:,[0, 2, 4, 8, 9]])
y_trigram = np.array(t_df.loc[t_df['previous_previous_move_duration'] > 0].iloc[:, 7]).ravel()

In [36]:
# This function is used to construct the javascript function for the
#   Decision Tree regressor, for potential use in the web application
def tree_to_code_javascript(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("function tree({}) {{".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        mini_indent = "  " * (depth-1)
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if ({} <= {}) {{".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
#            print ("{}else {{ // if {} > {}".format(indent, name, threshold))
            print ("{}else {{".format(indent))
            recurse(tree_.children_right[node], depth + 1)
            print ("{}}}".format(mini_indent))
        else:
            print ("{}return {} \n{}}}".format(indent, tree_.value[node][0][0], mini_indent))

    recurse(0, 1)

In [37]:
# Function to repeat training & testing the regressor
# using set random seeds (0 to N-1)
# which is to ensure same sets of randomized train & test datasets are used
# for all models
def evaluate_regressor(regressor, X, y, N=100, name="Regressor"):
  r_sq = []
  rmse = []
  for n in range(N):
    print("Progress: " + str(n) + "th run out of " + str(N-1) + \
          ". Random seed=" + str(n))
    tf.keras.utils.set_random_seed(n)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    r_sq.append(np.corrcoef(y_test, y_pred)[0,1]**2)
    rmse.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    if n==special_seed_val:
      print("This particular iteration:")
      show_pred_vs_actual(regressor, X_test, y_test, name)
      print("(Evaluated for single run with random seed="+str(n)+")")

  print("Average computed over " + str(N) + " runs")
  print("R-squared - avg:",np.mean(r_sq),"std:",np.std(r_sq))
  print("RMSE - avg:",np.mean(rmse),"std:",np.std(rmse))

In [38]:
regressor_DT = DecisionTreeRegressor()
evaluate_regressor(regressor_DT, X, y, N=reg_runs,
                   name="Decision Tree Regressor")

Progress: 0th run out of 39. Random seed=0
Progress: 1th run out of 39. Random seed=1
Progress: 2th run out of 39. Random seed=2
Progress: 3th run out of 39. Random seed=3
Progress: 4th run out of 39. Random seed=4
Progress: 5th run out of 39. Random seed=5
This particular iteration:


R-squared 0.7481794396318574
RMSE 100.59272646514792
(Evaluated for single run with random seed=5)
Progress: 6th run out of 39. Random seed=6
Progress: 7th run out of 39. Random seed=7
Progress: 8th run out of 39. Random seed=8
Progress: 9th run out of 39. Random seed=9
Progress: 10th run out of 39. Random seed=10
Progress: 11th run out of 39. Random seed=11
Progress: 12th run out of 39. Random seed=12
Progress: 13th run out of 39. Random seed=13
Progress: 14th run out of 39. Random seed=14
Progress: 15th run out of 39. Random seed=15
Progress: 16th run out of 39. Random seed=16
Progress: 17th run out of 39. Random seed=17
Progress: 18th run out of 39. Random seed=18
Progress: 19th run out of 39. Random seed=19
Progress: 20th run out of 39. Random seed=20
Progress: 21th run out of 39. Random seed=21
Progress: 22th run out of 39. Random seed=22
Progress: 23th run out of 39. Random seed=23
Progress: 24th run out of 39. Random seed=24
Progress: 25th run out of 39. Random seed=25
Progress:

In [39]:
regressor_RF = RandomForestRegressor(n_estimators=300)
evaluate_regressor(regressor_RF, X, y, N=reg_runs,
                   name="Random Forest Regressor")

Progress: 0th run out of 39. Random seed=0
Progress: 1th run out of 39. Random seed=1
Progress: 2th run out of 39. Random seed=2
Progress: 3th run out of 39. Random seed=3
Progress: 4th run out of 39. Random seed=4
Progress: 5th run out of 39. Random seed=5
This particular iteration:


R-squared 0.8007546868512361
RMSE 85.66004403620191
(Evaluated for single run with random seed=5)
Progress: 6th run out of 39. Random seed=6
Progress: 7th run out of 39. Random seed=7
Progress: 8th run out of 39. Random seed=8
Progress: 9th run out of 39. Random seed=9
Progress: 10th run out of 39. Random seed=10
Progress: 11th run out of 39. Random seed=11
Progress: 12th run out of 39. Random seed=12
Progress: 13th run out of 39. Random seed=13
Progress: 14th run out of 39. Random seed=14
Progress: 15th run out of 39. Random seed=15
Progress: 16th run out of 39. Random seed=16
Progress: 17th run out of 39. Random seed=17
Progress: 18th run out of 39. Random seed=18
Progress: 19th run out of 39. Random seed=19
Progress: 20th run out of 39. Random seed=20
Progress: 21th run out of 39. Random seed=21
Progress: 22th run out of 39. Random seed=22
Progress: 23th run out of 39. Random seed=23
Progress: 24th run out of 39. Random seed=24
Progress: 25th run out of 39. Random seed=25
Progress: 

In [40]:
regressor_KNN = KNeighborsRegressor(n_neighbors=7, metric='minkowski', p=2,
                      metric_params={'w': np.array([5,1,1])})
evaluate_regressor(regressor_KNN, X, y, N=reg_runs,
                   name="K-Nearest Neighbors Regressor")

Progress: 0th run out of 39. Random seed=0
Progress: 1th run out of 39. Random seed=1
Progress: 2th run out of 39. Random seed=2
Progress: 3th run out of 39. Random seed=3
Progress: 4th run out of 39. Random seed=4
Progress: 5th run out of 39. Random seed=5
This particular iteration:


R-squared 0.8158236562551954
RMSE 77.67814850133887
(Evaluated for single run with random seed=5)
Progress: 6th run out of 39. Random seed=6
Progress: 7th run out of 39. Random seed=7
Progress: 8th run out of 39. Random seed=8
Progress: 9th run out of 39. Random seed=9
Progress: 10th run out of 39. Random seed=10
Progress: 11th run out of 39. Random seed=11
Progress: 12th run out of 39. Random seed=12
Progress: 13th run out of 39. Random seed=13
Progress: 14th run out of 39. Random seed=14
Progress: 15th run out of 39. Random seed=15
Progress: 16th run out of 39. Random seed=16
Progress: 17th run out of 39. Random seed=17
Progress: 18th run out of 39. Random seed=18
Progress: 19th run out of 39. Random seed=19
Progress: 20th run out of 39. Random seed=20
Progress: 21th run out of 39. Random seed=21
Progress: 22th run out of 39. Random seed=22
Progress: 23th run out of 39. Random seed=23
Progress: 24th run out of 39. Random seed=24
Progress: 25th run out of 39. Random seed=25
Progress: 

In [41]:
#tree_to_code_javascript(regressor_DT,["move_no", "day_no", "time_no"])

In [42]:
# INITIAL ANN RUNS
# Train the model with N=ANN_runs number of randomized data splits
# with 500 epochs per run
# The purpose of this set of runs is to find the optimal number of epochs
#   for each random seed (0 to N-1)
history_list = []
for i in range(ANN_runs):
  print("Progress: " + str(i) + "th run out of " + str(ANN_runs-1) + \
        ". Random seed=" + str(i))
  tf.keras.utils.set_random_seed(i)

  scaler = StandardScaler()

  X_train_full, X_test, y_train_full, y_test = train_test_split(X_trigram, y_trigram, test_size=0.3)
  X_valid, X_train = X_train_full[:int(len(X_train_full)/2)], X_train_full[int(len(X_train_full)/2):]
  y_valid, y_train = y_train_full[:int(len(X_train_full)/2)], y_train_full[int(len(X_train_full)/2):]
  X_train = scaler.fit_transform(X_train)
  X_valid = scaler.transform(X_valid)
  X_test = scaler.transform(X_test)

  model = keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape=[5]))
  model.add(keras.layers.Dense(256, activation="tanh"))
  model.add(keras.layers.Dense(64, activation="tanh"))
  model.add(keras.layers.Dense(1, activation="relu"))
  model.compile(loss="mean_squared_error",
                optimizer="sgd")
  history_list.append(model.fit(X_train, y_train, epochs=500, verbose=0,
                                validation_data=(X_valid, y_valid)))

Progress: 0th run out of 19. Random seed=0
Progress: 1th run out of 19. Random seed=1
Progress: 2th run out of 19. Random seed=2
Progress: 3th run out of 19. Random seed=3
Progress: 4th run out of 19. Random seed=4
Progress: 5th run out of 19. Random seed=5
Progress: 6th run out of 19. Random seed=6
Progress: 7th run out of 19. Random seed=7
Progress: 8th run out of 19. Random seed=8
Progress: 9th run out of 19. Random seed=9
Progress: 10th run out of 19. Random seed=10
Progress: 11th run out of 19. Random seed=11
Progress: 12th run out of 19. Random seed=12
Progress: 13th run out of 19. Random seed=13
Progress: 14th run out of 19. Random seed=14
Progress: 15th run out of 19. Random seed=15
Progress: 16th run out of 19. Random seed=16
Progress: 17th run out of 19. Random seed=17
Progress: 18th run out of 19. Random seed=18
Progress: 19th run out of 19. Random seed=19


In [43]:
# Look inside history list for epoch numbers with good value losses
good_loss_indices = []
for i in range(ANN_runs):
  val_loss_list = history_list[i].history["val_loss"]
  index_of_lowest_val_loss = val_loss_list.index(min(val_loss_list))+1
  good_loss_indices.append(index_of_lowest_val_loss)

In [44]:
# FINAL ANN RUNS
# The purpose of this set of runs is to train the model to optimality
#   for each random seed (0 to N-1) with a set number of epochs determined above

# Store the evaluation metrics for each trained model
rsq_list = []
rmse_list = []

for i in range(ANN_runs):
  print("Progress: " + str(i) + "th run out of " + str(ANN_runs-1) + \
        ". Random seed=" + str(i))
  tf.keras.utils.set_random_seed(i)

  scaler = StandardScaler()

  X_train_full, X_test, y_train_full, y_test = train_test_split(X_trigram, y_trigram, test_size=0.3)
  X_valid, X_train = X_train_full[:int(len(X_train_full)/2)], X_train_full[int(len(X_train_full)/2):]
  y_valid, y_train = y_train_full[:int(len(X_train_full)/2)], y_train_full[int(len(X_train_full)/2):]
  X_train = scaler.fit_transform(X_train)
  X_valid = scaler.transform(X_valid)
  X_test = scaler.transform(X_test)

  model = keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape=[5]))
  model.add(keras.layers.Dense(256, activation="tanh"))
  model.add(keras.layers.Dense(64, activation="tanh"))
  model.add(keras.layers.Dense(1, activation="relu"))


  model.compile(loss="mean_squared_error",
                optimizer="sgd")
  model.fit(X_train, y_train, epochs=good_loss_indices[i], verbose=0,
            validation_data=(X_valid, y_valid))

  if i==special_seed_val:
    model.summary()
    print("This particular iteration:")
    show_pred_vs_actual_ANN(model, X_test, y_test, "ANN Predictive Model")
    print("(Evaluated for single run with random seed="+str(i)+")")

  y_pred = model.predict(X_test)

  rsq_list.append(np.corrcoef(y_test, y_pred[:,0])[0,1]**2)
  rmse_list.append(np.sqrt(mean_squared_error(y_test, y_pred[:,0])))

Progress: 0th run out of 19. Random seed=0
5/5 [==============================] - 0s 3ms/step
Progress: 1th run out of 19. Random seed=1
5/5 [==============================] - 0s 3ms/step
Progress: 2th run out of 19. Random seed=2
5/5 [==============================] - 0s 4ms/step
Progress: 3th run out of 19. Random seed=3
5/5 [==============================] - 0s 3ms/step
Progress: 4th run out of 19. Random seed=4
5/5 [==============================] - 0s 3ms/step
Progress: 5th run out of 19. Random seed=5
Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_25 (Flatten)        (None, 5)                 0         
                                                                 
 dense_75 (Dense)            (None, 256)               1536      
                                                                 
 dense_76 (Dense)            (None, 64)                16448     
  

R-squared 0.6123770224349921
RMSE 104.23684602325619
(Evaluated for single run with random seed=5)
5/5 [==============================] - 0s 3ms/step
Progress: 6th run out of 19. Random seed=6
5/5 [==============================] - 0s 3ms/step
Progress: 7th run out of 19. Random seed=7
5/5 [==============================] - 0s 3ms/step
Progress: 8th run out of 19. Random seed=8
5/5 [==============================] - 0s 3ms/step
Progress: 9th run out of 19. Random seed=9
5/5 [==============================] - 0s 4ms/step
Progress: 10th run out of 19. Random seed=10
5/5 [==============================] - 0s 3ms/step
Progress: 11th run out of 19. Random seed=11
5/5 [==============================] - 0s 4ms/step
Progress: 12th run out of 19. Random seed=12
5/5 [==============================] - 0s 4ms/step
Progress: 13th run out of 19. Random seed=13
5/5 [==============================] - 0s 3ms/step
Progress: 14th run out of 19. Random seed=14
5/5 [==============================] - 0s 3ms

In [45]:
print("ANN Evaluation")
print("Average computed over " + str(ANN_runs) + " runs")
print(np.mean(rsq_list), np.std(rsq_list))
print(np.mean(rmse_list), np.std(rmse_list))

ANN Evaluation
Average computed over 20 runs
0.5491822988033435 0.08182805016187997
122.05228810599367 11.755087507423202
